<a href="https://colab.research.google.com/github/B0BWAX/MT5-TRANSLATION-FINETUNING/blob/main/MT5_TRANSLATION_FINETUNING.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install datasets transformers sentencepiece sacrebleu

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 542.0/542.0 kB 5.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 106.7/106.7 kB 9.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 9.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.1/194.1 kB 7.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 9.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 388.9/388.9 kB 12.7 MB/s eta 0:00:00
  Attempting uninstall: huggingface-hub
    Found existing installation: huggingface-hub 0.20.3
    Uninstalling huggingface-hub-0.20.3:
      Successfully uninstalled huggingface-hub-0.20.3


In [2]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


## Load Model and Tokenizer

Model: [Google MT5-Small](https://huggingface.co/google/mt5-small)

In [8]:
model_checkpoint = "google/mt5-small"

In [7]:
from transformers import AutoTokenizer
tokenizer = AutoTokenizer.from_pretrained(model_checkpoint)

/usr/local/lib/python3.10/dist-packages/transformers/convert_slow_tokenizer.py:560: UserWarning: The sentencepiece tokenizer that you are converting to a fast tokenizer uses the byte fallback option which is not implemented in the fast tokenizers. In practice this means that the fast version of the tokenizer can produce unknown tokens whereas the sentencepiece version would have converted these unknown tokens into a sequence of byte tokens matching the original piece of text.
  warnings.warn(


In [9]:
from transformers import AutoModelForSeq2SeqLM
model = AutoModelForSeq2SeqLM.from_pretrained(model_checkpoint)
model = model.cuda()

pytorch_model.bin:   0%|          | 0.00/1.20G [00:00<?, ?B/s]

## Load Dataset

Translation Dataset: [XNLI](https://huggingface.co/datasets/alvations/xnli-15way?row=1)

In [41]:
import datasets
from datasets import load_dataset, load_metric, Dataset

In [11]:
dataset = load_dataset("alvations/xnli-15way")
metric = load_metric("sacrebleu")

Repo card metadata block was not found. Setting CardData to empty.


Generating train split:   0%|          | 0/10001 [00:00<?, ? examples/s]

<ipython-input-11-aa160c1170ae>:2: FutureWarning: load_metric is deprecated and will be removed in the next major version of datasets. Use 'evaluate.load' instead, from the new library 🤗 Evaluate: https://huggingface.co/docs/evaluate
  metric = load_metric("sacrebleu")
/usr/local/lib/python3.10/dist-packages/datasets/load.py:759: FutureWarning: The repository for sacrebleu contains custom code which must be executed to correctly load the metric. You can inspect the repository content at https://raw.githubusercontent.com/huggingface/datasets/2.19.0/metrics/sacrebleu/sacrebleu.py
You can avoid this message in future by passing the argument `trust_remote_code=True`.
Passing `trust_remote_code=True` will be mandatory to load this metric from the next major release of `datasets`.
  warnings.warn(


In [12]:
dataset

DatasetDict({
    train: Dataset({
        features: ['text'],
        num_rows: 10001
    })
})

In [13]:
# since dataset only has 1 split we must create training, testing, and validation sets
train_testvalid = dataset['train'].train_test_split(test_size=0.2)
test_valid = train_testvalid['test'].train_test_split(test_size=0.5)

dataset = datasets.DatasetDict({
    'train': train_testvalid['train'],
    'test': test_valid['test'],
    'valid': test_valid['train']
})

In [14]:
dataset

DatasetDict({
    train: Dataset({
        features: ['text'],
        num_rows: 8000
    })
    test: Dataset({
        features: ['text'],
        num_rows: 1001
    })
    valid: Dataset({
        features: ['text'],
        num_rows: 1000
    })
})

In [15]:
dataset["train"][0]

{'text': "لم أكن أبدا في الأنشطة الخاصة .\tНикога не съм бил в Специални дейности .\tIch war niemals in Special Activities .\tΠοτέ δεν ήμουν σε Ειδικές Δραστηριότητες .\tI was never in Special Activities .\tNunca estuve en actividades especiales .\tJe n' ai jamais participé à des Activités Spéciales .\tमैं विशेष गतिविधियों में कभी भी नहीं था ।\tЯ никогда не был в спецподразделениях .\tSijawahi kuwa katika Special Activities .\tฉันไม ่ เคยร ่ วมกิจกรรมพิเศษ\tBen hiçbir zaman Özel Aktivitelerde olmadım .\tمیں خاص سرگرمیوں کا حصہ کبھی نہيں تھا ۔\tTôi chưa bao giờ trong đội Hoạt động Đặc biệt .\t我 从未 参加 特别 活动 。"}

The order of the data is "ar bg de el en es fr hi ru sw th tr ur vi zh" in a tab seperated format (.tsv) to simplify usage we can convert it to a dictionary format.

In [16]:
def tsv_to_json(example):
    example["text"] = {lang: text for lang, text in zip(['ar', 'bg', 'de', 'el', 'en', 'es', 'fr', 'hi', 'ru', 'sw', 'th', 'tr', 'ur', 'vi', 'zh'], example["text"].split('\t'))}
    return example

In [17]:
dataset = dataset.map(tsv_to_json)

Map:   0%|          | 0/8000 [00:00<?, ? examples/s]

Map:   0%|          | 0/1001 [00:00<?, ? examples/s]

Map:   0%|          | 0/1000 [00:00<?, ? examples/s]

In [50]:
train = Dataset.from_list(dataset["train"])
validation = Dataset.from_list(dataset['valid'])
test = Dataset.from_list(dataset['test'])

In [51]:
train[2]

{'text': {'ar': 'لم نفعل أي شيء لعيد ميلادها .',
  'bg': 'За нейния рожден ден не правихме нищо специално .',
  'de': 'Wir haben nichts für ihren Geburtstag getan .',
  'el': 'Δεν κάναμε τίποτα για τα γενέθλιά της .',
  'en': 'We didnt do anything for her birthday .',
  'es': 'No hicimos nada por su cumpleaños .',
  'fr': "Nous n' avons rien fait pour son anniversaire .",
  'hi': 'हमने उसके जन ् मदिन के लिए कुछ भी नहीं किया ।',
  'ru': 'Мы ничего не делали на ее день рождения .',
  'sw': 'Hatukufanya chochote kwa siku yake ya kuzaliwa .',
  'th': 'เราไม ่ ได ้ ทำอะไรเพื ่ อวันเกิดของเธอเลย',
  'tr': 'Onun doğum günü için hiç bir şey yapmadık .',
  'ur': 'ہم نے اس کے جنم دن کے لیے کچھ بھی نہیں کیا ۔',
  'vi': 'Chúng tôi không làm gì cho sinh nhật của cô ấy cả .',
  'zh': '我们 没有 庆祝 她 的 生日 。'}}

## Preprocessing

We will fine tune the model for the following languages:
* English
* French
* Spanish
* German

In [21]:
max_seq_len = 32

In [22]:
LANG_TOKEN_MAPPING = {
    'en': '<en>',
    'fr': '<fr>',
    'es': '<es>',
    'de': '<de>'
}

In [23]:
# special tokens
special_tokens_dict = {'additional_special_tokens': list(LANG_TOKEN_MAPPING.values())}
tokenizer.add_special_tokens(special_tokens_dict)
model.resize_token_embeddings(len(tokenizer))

Embedding(250104, 512)

In [52]:
import numpy as np
import torch

def encode_input_str(text, target_lang, tokenizer, seq_len,
                     lang_token_map=LANG_TOKEN_MAPPING):
  target_lang_token = lang_token_map[target_lang]

  # Tokenize and add special tokens
  input_ids = tokenizer.encode(
      text = target_lang_token + text,
      return_tensors = 'pt',
      padding = 'max_length',
      truncation = True,
      max_length = seq_len)

  return input_ids[0]

def encode_target_str(text, tokenizer, seq_len,
                      lang_token_map=LANG_TOKEN_MAPPING):
  token_ids = tokenizer.encode(
      text = text,
      return_tensors = 'pt',
      padding = 'max_length',
      truncation = True,
      max_length = seq_len)

  return token_ids[0]

def format_translation_data(translations, lang_token_map,
                            tokenizer, seq_len=128):
  # Choose a random 2 languages for in i/o
  langs = list(lang_token_map.keys())
  input_lang, target_lang = np.random.choice(langs, size=2, replace=False)

  # Get the translations for the batch
  input_text = translations[input_lang]
  target_text = translations[target_lang]

  if input_text is None or target_text is None:
    return None

  input_token_ids = encode_input_str(
      input_text, target_lang, tokenizer, seq_len, lang_token_map)

  target_token_ids = encode_target_str(
      target_text, tokenizer, seq_len, lang_token_map)

  return input_token_ids, target_token_ids

def transform_batch(batch, lang_token_map, tokenizer):
  inputs = []
  targets = []
  for translation_set in batch['text']:
    formatted_data = format_translation_data(
        translation_set, lang_token_map, tokenizer, max_seq_len)

    if formatted_data is None:
      continue

    input_ids, target_ids = formatted_data
    inputs.append(input_ids.unsqueeze(0))
    targets.append(target_ids.unsqueeze(0))

  batch_input_ids = torch.cat(inputs).cuda()
  batch_target_ids = torch.cat(targets).cuda()

  return batch_input_ids, batch_target_ids

def get_data_generator(dataset, lang_token_map, tokenizer, batch_size=32):
  dataset = dataset.shuffle()
  for i in range(0, len(dataset), batch_size):
    raw_batch = dataset[i:i+batch_size]
    yield transform_batch(raw_batch, lang_token_map, tokenizer)

In [54]:
# Testing `data_transform`
in_ids, out_ids = format_translation_data(
    train[0]['text'], LANG_TOKEN_MAPPING, tokenizer)

print(' '.join(tokenizer.convert_ids_to_tokens(in_ids)))
print(' '.join(tokenizer.convert_ids_to_tokens(out_ids)))

# Testing data generator
data_gen = get_data_generator(train, LANG_TOKEN_MAPPING, tokenizer, 8)
data_batch = next(data_gen)
print('Input shape:', data_batch[0].shape)
print('Output shape:', data_batch[1].shape)

<de> ▁Je ▁ n ' ▁ai ▁ jamais ▁particip é ▁ à ▁des ▁A ctivités ▁S pécial es ▁ . </s> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad>
▁Ich ▁war ▁nie mals ▁in ▁Special ▁Activities ▁ . </s> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad>

## Finetuning

In [55]:
from transformers import AdamW, get_linear_schedule_with_warmup

n_epochs = 7
batch_size = 10
print_freq = 100
checkpoint_freq = 500
lr = 5e-4
n_batches = int(np.ceil(len(train) / batch_size))
total_steps = n_epochs * n_batches
n_warmup_steps = int(total_steps * 0.01)

optimizer = AdamW(model.parameters(), lr=lr)
scheduler = get_linear_schedule_with_warmup(
    optimizer, n_warmup_steps, total_steps)

losses = []

/usr/local/lib/python3.10/dist-packages/transformers/optimization.py:521: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  warnings.warn(


In [36]:
def eval_model(model, gdataset, max_iters=8):
  test_generator = get_data_generator(gdataset, LANG_TOKEN_MAPPING,
                                      tokenizer, batch_size)
  eval_losses = []
  for i, (input_batch, label_batch) in enumerate(test_generator):
    if i >= max_iters:
      break

    model_out = model.forward(
        input_ids = input_batch,
        labels = label_batch)
    eval_losses.append(model_out.loss.item())

  return np.mean(eval_losses)

In [65]:
from tqdm import tqdm_notebook

for epoch_idx in range(n_epochs):
  # Randomize data order
  data_generator = get_data_generator(train, LANG_TOKEN_MAPPING,
                                      tokenizer, batch_size)

  for batch_idx, (input_batch, label_batch) \
      in tqdm_notebook(enumerate(data_generator), total=n_batches):
    optimizer.zero_grad()

    # Forward pass
    model_out = model.forward(
        input_ids = input_batch,
        labels = label_batch)

    # Calculate loss and update weights
    loss = model_out.loss
    losses.append(loss.item())
    loss.backward()
    optimizer.step()
    scheduler.step()

    # Print training update info
    if (batch_idx + 1) % print_freq == 0:
      avg_loss = np.mean(losses[-print_freq:])
      print('Epoch: {} | Step: {} | Avg. loss: {:.3f} | lr: {}'.format(
          epoch_idx+1, batch_idx+1, avg_loss, scheduler.get_last_lr()[0]))

    if (batch_idx + 1) % checkpoint_freq == 0:
      validation_loss = eval_model(model, validation)
      print('Saving model with test loss of {:.3f}'.format(validation_loss))
      torch.save(model.state_dict(), '/content/drive/MyDrive/mt5_finetuned.pt')

torch.save(model.state_dict(), '/content/drive/MyDrive/mt5_finetuned.pt')

<ipython-input-65-9d918fff43a3>:9: TqdmDeprecationWarning: This function will be removed in tqdm==5.0.0
Please use `tqdm.notebook.tqdm` instead of `tqdm.tqdm_notebook`
  in tqdm_notebook(enumerate(data_generator), total=n_batches):


  0%|          | 0/800 [00:00<?, ?it/s]

Epoch: 1 | Step: 100 | Avg. loss: 0.812 | lr: 0.0001856962481962482
Epoch: 1 | Step: 200 | Avg. loss: 0.820 | lr: 0.00017667748917748918
Epoch: 1 | Step: 300 | Avg. loss: 0.799 | lr: 0.00016765873015873016
Epoch: 1 | Step: 400 | Avg. loss: 0.780 | lr: 0.00015863997113997114
Epoch: 1 | Step: 500 | Avg. loss: 0.794 | lr: 0.00014962121212121213
Saving model with test loss of 0.999
Epoch: 1 | Step: 600 | Avg. loss: 0.798 | lr: 0.00014060245310245308
Epoch: 1 | Step: 700 | Avg. loss: 0.775 | lr: 0.0001315836940836941
Epoch: 1 | Step: 800 | Avg. loss: 0.753 | lr: 0.00012256493506493507


  0%|          | 0/800 [00:00<?, ?it/s]

Epoch: 2 | Step: 100 | Avg. loss: 0.787 | lr: 0.00011354617604617605
Epoch: 2 | Step: 200 | Avg. loss: 0.719 | lr: 0.00010452741702741703
Epoch: 2 | Step: 300 | Avg. loss: 0.749 | lr: 9.550865800865801e-05
Epoch: 2 | Step: 400 | Avg. loss: 0.744 | lr: 8.6489898989899e-05
Epoch: 2 | Step: 500 | Avg. loss: 0.745 | lr: 7.747113997113998e-05
Saving model with test loss of 0.924
Epoch: 2 | Step: 600 | Avg. loss: 0.728 | lr: 6.845238095238096e-05
Epoch: 2 | Step: 700 | Avg. loss: 0.731 | lr: 5.943362193362194e-05
Epoch: 2 | Step: 800 | Avg. loss: 0.703 | lr: 5.041486291486291e-05


...

Epoch: 5 | Step: 100 | Avg. loss: 0.701 | lr: 0.0
Epoch: 5 | Step: 200 | Avg. loss: 0.666 | lr: 0.0
Epoch: 5 | Step: 300 | Avg. loss: 0.669 | lr: 0.0
Epoch: 5 | Step: 400 | Avg. loss: 0.693 | lr: 0.0
Epoch: 5 | Step: 500 | Avg. loss: 0.681 | lr: 0.0
Saving model with test loss of 1.039
Epoch: 5 | Step: 600 | Avg. loss: 0.730 | lr: 0.0
Epoch: 5 | Step: 700 | Avg. loss: 0.681 | lr: 0.0
Epoch: 5 | Step: 800 | Avg. loss: 0.677 | lr: 0.0


  0%|          | 0/800 [00:00<?, ?it/s]

Epoch: 6 | Step: 100 | Avg. loss: 0.662 | lr: 0.0
Epoch: 6 | Step: 200 | Avg. loss: 0.698 | lr: 0.0
Epoch: 6 | Step: 300 | Avg. loss: 0.685 | lr: 0.0
Epoch: 6 | Step: 400 | Avg. loss: 0.673 | lr: 0.0
Epoch: 6 | Step: 500 | Avg. loss: 0.696 | lr: 0.0
Saving model with test loss of 1.024
Epoch: 6 | Step: 600 | Avg. loss: 0.656 | lr: 0.0
Epoch: 6 | Step: 700 | Avg. loss: 0.667 | lr: 0.0
Epoch: 6 | Step: 800 | Avg. loss: 0.686 | lr: 0.0


  0%|          | 0/800 [00:00<?, ?it/s]

Epoch: 7 | Step: 100 | Avg. loss: 0.656 | lr: 0.0
Epoch: 7 | Step: 200 | Avg. loss: 0.674 | lr: 0.0
Epoch: 7 | Step: 300 | Avg. loss: 0.693 | lr: 0.0
Epoch: 7 | Step: 400 | Avg. loss: 0.693 | lr: 0.0
Epoch: 7 | Step: 500 | Avg. loss: 0.693 | lr: 0.0
Saving model with test loss of 0.980
Epoch: 7 | Step: 600 | Avg. loss: 0.699 | lr: 0.0
Epoch: 7 | Step: 700 | Avg. loss: 0.700 | lr: 0.0
Epoch: 7 | Step: 800 | Avg. loss: 0.685 | lr: 0.0


In [73]:
#@title Test Interface
input_text = 'the capital of Egypt is Cairo' #@param {type:"string"}
output_language = 'es' #@param ["en", "de", "fr", "es"]

input_ids = encode_input_str(
    text = input_text,
    target_lang = output_language,
    tokenizer = tokenizer,
    seq_len = 128,
    lang_token_map = LANG_TOKEN_MAPPING)
input_ids = input_ids.unsqueeze(0).cuda()

output_tokens = model.generate(input_ids, num_beams=20, length_penalty=0.2)
print(input_text + '  ->  ' + \
      tokenizer.decode(output_tokens[0], skip_special_tokens=True))

the capital of Egypt is Cairo  ->  El capital de Egipto es Cairo.
